In [ ]:
REQUEST_ID = 0
AOI = 'POLYGON ((-85.483932 40.367801, -85.64349199999999 40.367801, -85.64349199999999 40.226597, -85.483932 40.226597, -85.483932 40.367801))'
START_DATE = '2020-04-01'
END_DATE = '2020-09-01'

In [ ]:
import shapely.wkt
import time
import os
import shutil
import sys
import yaml
import json
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from prediction_pipeline.run import run

In [ ]:
USER = os.getenv('NB_USER')
NAME = 'Crop map'
BASE = os.getcwd()
DATA_DIR = f"/home/{USER}/work"
RESULTS_DIR = os.path.join(DATA_DIR, "results/pw/")

In [ ]:
polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")
start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
end_date = datetime.strptime(END_DATE, '%Y-%m-%d')

In [ ]:
CONFIG_PATH = f"prediction_pipeline/config.yml"
with open(CONFIG_PATH) as f:
    config = yaml.safe_load(f)
    
config['input']['data_dir'] = os.path.join(DATA_DIR, 'notebooks/pw/data/predictions')
config['sentinel1']['download']['scihub_creds'] = os.path.join(BASE, '.secret/scihub_creds.json')
config['sentinel2']['download']['google_api_key'] = os.path.join(BASE, '.secret/sentinel2_google_api_key.json')
config['predict']['model_weights_path'] = os.path.join(BASE, 'data/experiments/mi_clstm_192_mask_64_crop_s1_s2_5_months_10m/logs/checkpoints/best.pth')
config['predict']['experiments_dir'] = os.path.join(BASE, 'data/experiments/')
config['predict']['class_names_json'] = os.path.join(BASE, 'data/labels/ohio/cropscape_labels/merged_classes/final_classes.json')

with open(CONFIG_PATH, 'w') as f:
    f.write(yaml.dump(config))

In [ ]:
prediction_file = run(CONFIG_PATH, aoi_filename, start_date, end_date, REQUEST_ID)
os.remove(aoi_filename)

In [ ]:
config

### Read prediction raster and mask with valid pixels 

In [ ]:
with rasterio.open(prediction_file) as src:
    img = src.read(1)
    mask = src.read_masks(1)
    profile = src.profile

## Creating metadata to be displayed on the frontend: legend and area calculation

In [ ]:
with open(CONFIG_PATH) as f:
    config = yaml.load(f)
    path_to_class_names = config['predict']['class_names_json']

if os.path.exists(path_to_class_names):
    with open(path_to_class_names) as f:
        class_names = json.load(f)
        class_names = ['Urban areas' if class_name == 'Developed' else class_name for class_name in class_names]
else:
    raise FileNotFoundError()

NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES
colors = plt.cm.jet(arr)
colors[0] = (0,0,0,1)

labels = []

for label, name in enumerate(class_names):
    class_area = len(img[img == label]) / 10 ** 4
    # convert list of float values into string representing color
    class_color = ",".join(list(map(lambda x: str(int(x)), colors[label][:-1] * 255)))
    if class_area != 0:
        labels.append({
            "color": class_color, 
            "name": name,
            "area": class_area
        })
    else:
        labels.append({
            "color": class_color, 
            "name": name
        })
labels = json.dumps(labels)
labels

### Scale class prediction raster and make it colored

In [ ]:
NUM_CLASSES = len(class_names)
nodata = 0
mask = mask.astype(bool)
scaled = img.astype(np.float32) / NUM_CLASSES
scaled = (plt.cm.jet(scaled)[:,:,:-1] * 255)
scaled[mask[:,:,np.newaxis] & (scaled==0)] += 1
scaled = np.clip(scaled, 0, 255).astype(np.uint8)
# Set pixels with invalid pixels to new nodata value
scaled[~mask] = nodata
# Set pixels with background class(0) to new nodata value
scaled[img==0] = nodata

### Update raster profile and save results

In [ ]:
prediction_dir = os.path.dirname(prediction_file)

profile.update(
    count=3,
    nodata=nodata,
    compress='lzw'
)
colored_tif = os.path.join(prediction_dir, "colored_prediction.tif")
with rasterio.open(colored_tif, 'w', **profile) as dst:
    dst.update_tags(start_date=START_DATE, 
                    end_date=END_DATE, 
                    request_id=REQUEST_ID,
                    labels=labels,
                    name=NAME)
    for i in range(scaled.shape[-1]):
        dst.write(scaled[:,:,i], indexes=i+1)

In [ ]:
result_name = f"{REQUEST_ID}_{START_DATE}_{END_DATE}.tif"
os.makedirs(RESULTS_DIR, exist_ok=True)
dst_file = os.path.join(RESULTS_DIR, result_name)
os.rename(colored_tif, dst_file)